# EDX Analysis for In-situ Heating Experiments from the Thermo Fisher Velox Software

This is a personal workbook used to process my own data but has been made public for the use of others. If you find any errors or need further help using this workbook please contact by email k8macarthur@gmail.com.

## Authors:

Katherine E. MacArthur: Originally written upon my leaving the Ernst Ruska Centre August 2021 for knowledge transfer.

## Requirements

This workbook was written with Hyperspy-base v1.6.3, numpy v1.20.2, matplotlib v3.4.2. Earlier versions than this may be possible but have not been tested.

## 1. Importing Libraries and Checking Parameters

We begin be importing the necessary libraries. 

It's also import to use this section to check the parameter selection (e.g. number of frames for EDX and number of components for PCA) before carrying out the full automated extraction below. 

In [1]:
%matplotlib qt
import matplotlib
import hyperspy.api as hs
import scipy.misc
import numpy as np
from PIL import Image

Loading_SI_image_stack=True means the whole ADF image stack is imported.

If a file_name is not provided then a window will pop up to allow selection of the file.


Including both a first_frame and last_frame allows to only select a portion of the EDX dataset in the time dimension. This should be used to test that enough signal can be reached with the desired time resolution (or window width in the time dimension.)

In [2]:
first_frame = 0
last_frame = 150
s = hs.load(first_frame=first_frame, last_frame =last_frame, load_SI_image_stack=True)

It is also important to check the order of the imported list. 

At the time of writing the exact position of the HAADF image (or image stack) changes for each data set. Even during when acquiring multiple datasets back-to-back with the same element settings. However the EDSSTEMSpectrum (integrating all detectors) is always at the end of the list and each of the ESDTEMSpectrum (individual) are at the start of the list.

In [3]:
s

[<EDSTEMSpectrum, title: EDS, dimensions: (|4096)>,
 <EDSTEMSpectrum, title: EDS, dimensions: (|4096)>,
 <EDSTEMSpectrum, title: EDS, dimensions: (|4096)>,
 <EDSTEMSpectrum, title: EDS, dimensions: (|4096)>,
 <Signal2D, title: HAADF, dimensions: (494|532, 928)>,
 <Signal2D, title: Pt, dimensions: (|532, 928)>,
 <Signal2D, title: Ni, dimensions: (|532, 928)>,
 <EDSTEMSpectrum, title: EDS, dimensions: (532, 928|4096)>]

In [4]:
#I like to store the ADF_location here for later use
ADF_location = 5
#Here we print the time resolution step so you can see the duration of the EDX map.
print('Time_step = ', s[ADF_location].axes_manager[0].scale*1000*(last_frame-first_frame)/60, 'min')

Time_step =  121.31483092617837 min


In [5]:
# This line will save the ADF image stack into a tif stack.
s[-2].save('1535_ADF_Stack.tif')

For the EDX data analsysis we being by plotting the raw intgrated data cube. 
Although at this stage it can be that we don't see a lot of signal in just the raw map.

Typically I crop the first 0.1eV from the start of the signal as I find there is a hardware peak there which messes up the intensity plotting of the map.

In [6]:
s = s[-1].isig[0.1:]
#Not this is now over writing our original s.
#but having saved the ADF stack we don't often have use for 
#the rest of the raw data cube so this helps to save memory.
s.plot()

Next it's important to adjust our rebin parameter. In my experience no high resolution EDX map can easily be processed without some binning. I am using the rebin function in conjunction with the plotting function to see the results. Once I'm happy with the number of bins I want to provide I can then go ahead and overwrite the signal again with a newly rebined signal.

A quick marker for the correct amount of binning is whether or not you can start to see a basic peak shape per pixel for the elements which will be quantified. The PCA analysis will later also help to show if the data has been sufficiently rebinned.

Obviously, if the spatial binning level needed is to high here you can go back and select a larger time dimension during import at the top.

In [7]:
s.rebin(scale=(15,15,2)).plot() 
#Here we are rebinning by 15 in both spatial dimensions and by 2 in the energy dimension.

In [8]:
s = s.rebin(scale=(15,15,2))
#This step saves and overwrites our spectrum data cube.

Next, because we're working on the low end of the signal-to-noise level I would always perfom seom PCA denoising. Again this should also be checked before we set up the automated analysis.

We start by performing a basic PCA decomposition and plotting the variance ratio.

Here we're looking for confirmation that the variance plot produces a clean elbow shaped graph that shows only a small number of clearly defined components (loadings) can be seen above the noise level.

In [9]:
s.change_dtype('float64')
s.decomposition()
s.plot_explained_variance_ratio()

Decomposition info:
  normalize_poissonian_noise=True
  algorithm=SVD
  output_dimension=None
  centre=None


<AxesSubplot:title={'center':'EDS\nPCA Scree Plot'}, xlabel='Principal component index', ylabel='Proportion of variance'>

From the PCA decomposition we now know the estimated number of components that could then be used for the denoising. For EDX analysis I prefer to use NMF (non-negative factorization) rather than PCA because this forces the restriction that all components must be strictly real and positive.

Once we have decomposed the data cube. We then reassemble the data cube with only the components that we consider above the noise. This technique is referred to as de-noising.

In [10]:
s.decomposition(algorithm='NMF', output_dimension=2)
#s.plot_explained_variance_ratio()
s.plot_decomposition_loadings(2)
sc = s.get_decomposition_model(2)
sc.plot()

/Users/macark/hyperspy-bundle/lib/python3.8/site-packages/hyperspy/learn/mva.py:271: VisibleDeprecationWarning: The algorithm name `nmf` has been deprecated and will be removed in HyperSpy 2.0. Please use `NMF` instead.
  warnings.warn(
/Users/macark/hyperspy-bundle/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


Decomposition info:
  normalize_poissonian_noise=True
  algorithm=NMF
  output_dimension=2
  centre=None
scikit-learn estimator:
NMF(n_components=2)


If you're not sure exactly what number of components to use for the reconstructed EDX data cube, you can look at the difference between 'n' and 'n+1' components check there is only residual noise difference and nothing structural.

In [165]:
s.decomposition(algorithm='nmf', output_dimension=3)
sc2 = s.get_decomposition_model(3)
(sc2-sc).plot()

/Users/macark/Documents/Git/hyperspy/hyperspy/drawing/utils.py:133: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(**kwargs)
/Users/macark/Documents/Git/hyperspy/hyperspy/drawing/utils.py:133: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(**kwargs)


Now we check the metadata and set the require elements for extraction of maps.

In [12]:
#sc.add_elements(['C'])
sc.set_lines(['Ni_Ka', 'Pt_Lb1', 'Pt_Lb2'])
sc.plot(True)

Next we get the X-ray lines and extract them. Here we're not yet applying a background subtraction.

In [14]:
lines = sc.get_lines_intensity()
lines

[<BaseSignal, title: X-ray line intensity of EDS model from decomposition with 2 components: Ni_Ka at 7.48 keV, dimensions: (35, 61|)>,
 <BaseSignal, title: X-ray line intensity of EDS model from decomposition with 2 components: Pt_Lb1 at 11.07 keV, dimensions: (35, 61|)>,
 <BaseSignal, title: X-ray line intensity of EDS model from decomposition with 2 components: Pt_Lb2 at 11.25 keV, dimensions: (35, 61|)>]

The next lines of code can be used to plot each of the X-ray maps in your list of maps. The save function also saves these for later if needed.

In [168]:
for line in lines:
    line.plot()
    Image.fromarray(line.data).save('1534_'+line.metadata.Sample.xray_lines[0]+'.tif') 

## 2. Running the main EDX In-situ Analysis

Having checked the parameters we can now run a full in-situ analysis for an EDX map.

Beginning with setting the parameters.

In [81]:
window_range = 250 #How many ADF frames do you want include into each EDX map.
overlap = 100 #In order to maintain time resolution you can add an additional overlap
no_of_frames_1 = 442 #This can be determined from the data but is easier if set here after checking once.
file_name_1 = '1406 EDS-HAADF 2.0 Mx.emd' #The file name is needed as we'll be loading many times and 
                                           #don't want to have to use the interactive window each time.

file_location = '/Users/macark/Desktop/TUB-EDX-PtNiX/In-Situ_Heating_Feb20/300C/'
save_folder = '1406'
adf_location = 5
bin_scale = (8,8,2)
no_of_components = 3 #for nmf decomposition

#Parameters for quantification.
beam_current = 0.06466 #in nA
elements = ['Pt', 'Ni']
lines = ['Ni_Ka', 'Ni_Kb', 'Pt_Lb1', 'Pt_Lb2']

In [67]:
s[adf_location].inav[191:441]

<Signal2D, title: HAADF, dimensions: (250|532, 928)>

If using the quantification part of the code then you need to import or select calibrations the follow steps do this for EDX cross_sections.

In [77]:
calibration = hs.load('ChemiSTEMCal_80kV_15.04.20.hspy').metadata.calibration
cs = (((calibration['Ni_Ka']['cross_section'] + calibration['Ni_Kb']['cross_section']),
       (calibration['Ni_Ka']['cross_section'] + calibration['Ni_Kb']['cross_section']),
       (calibration['Pt_La']['cross_section'] + calibration['Pt_Lb1']['cross_section']),
       (calibration['Pt_La']['cross_section'] + calibration['Pt_Lb1']['cross_section'])))
iw = calibration['Ni_Ka']['Integration windows'] + calibration['Pt_La']['Integration windows']
bw = np.append(calibration['Ni_Ka']['Background windows'], 
               calibration['Pt_La']['Background windows'],
                  axis=0)

Before running the next section of code you will need to run the two functions which are stored at the bottom of the work book. It's worth noting here that the open_and_save function will run and save count maps for any number of elements defined in the list above. The quantify and save function was only written for the Pt Ni binary investigations and will need to modified to include other elements.

In [83]:
i=0
height = int(np.ceil(no_of_frames_1/(window_range-overlap)))
shape = (7, 116, 66) #Again this can be specifically determined but I find it easier to just set it manually.
#First number is the number of frames you will get after processing and then x-dimension, y-dimension).
Pt_counts = np.zeros(shape)
Ni_counts = np.zeros(shape)

quantify = True
threshold = 10 #needed for making composition maps of nanoparticles.

if quantify == True:
    Ni_atoms = np.zeros(shape)
    Pt_atoms = np.zeros(shape)
    
    Ni_composition = np.zeros(shape)
    Pt_composition = np.zeros(shape)
    
i=0

while (i * overlap + window_range) < no_of_frames_1:

    sc, maps = open_and_save_maps(save_folder+'/',
                              file_location + file_name_1, 
                              start_window=i*overlap, 
                              end_window=((i*overlap)+window_range),
                              ADF_location=adf_location,
                              beam_current=beam_current,
                              elements=elements,
                              lines=lines,
                              save=True,
                              no_of_components = no_of_components,
                              bin_scale = bin_scale,
                              iw=None, 
                              bw=None)

    
    Pt_counts[i] = (maps[1] + maps[2]).data ##This part here needs to be modified to include all the elements you want to analyse.
    Ni_counts[i] = (maps[0]).data
    
    if quantify == True:
        results = quantify_and_save(save_folder+'/', 
                      sc, maps, cs, 
                      start_window=i*overlap, 
                      end_window=((i*overlap)+window_range),
                      threshold=10,
                      save=False)
    
        Ni_composition[int(i/(window_range-overlap))] = results[0].data
        Pt_composition[int(i/(window_range-overlap))] = results[1].data
        Ni_atoms[int(i/(window_range-overlap))] = results[2].data
        Pt_atoms[int(i/(window_range-overlap))] = results[3].data
    
    print ('i='+str(i)+'...done.')
    
    i += 1
    
sc, maps = open_and_save_maps (save_folder+'/',
                           file_location + file_name_1, 
                           start_window=(no_of_frames_1-1-window_range), 
                           end_window=(no_of_frames_1-1),
                           ADF_location=adf_location,
                           beam_current=beam_current,
                           elements=elements,
                           lines=lines,
                           save=True,
                           no_of_components = no_of_components,
                           bin_scale = bin_scale,
                           iw=None,
                           bw=None)

if quantify == True:
    results = quantify_and_save(save_folder+'/', 
                  sc, maps, cs, 
                  start_window=(no_of_frames_1-1-window_range), 
                  end_window=(no_of_frames_1-1),
                  threshold=10,
                  save=False)

    Ni_composition[int(i/(window_range-overlap))] = results[0].data
    Pt_composition[int(i/(window_range-overlap))] = results[1].data
    Ni_atoms[int(i/(window_range-overlap))] = results[2].data
    Pt_atoms[int(i/(window_range-overlap))] = results[3].data

Pt_counts[-1] = (maps[1] + maps[2]).data
Ni_counts[-1] = (maps[0]).data

<Signal2D, title: HAADF, dimensions: (442|532, 928)>
Decomposition info:
  normalize_poissonian_noise=True
  algorithm=SVD
  output_dimension=None
  centre=None


/Users/macark/hyperspy-bundle/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


Decomposition info:
  normalize_poissonian_noise=True
  algorithm=NMF
  output_dimension=7
  centre=None
scikit-learn estimator:
NMF(n_components=7)


/Users/macark/hyperspy-bundle/lib/python3.8/site-packages/hyperspy/misc/eds/utils.py:627: RuntimeWarning: invalid value encountered in true_divide
  composition = number_of_atoms / total_atoms


0-250 quantified
i=0...done.
<Signal2D, title: HAADF, dimensions: (442|532, 928)>
Decomposition info:
  normalize_poissonian_noise=True
  algorithm=SVD
  output_dimension=None
  centre=None


...

100-350 quantified
i=1...done.
<Signal2D, title: HAADF, dimensions: (442|532, 928)>
Decomposition info:
  normalize_poissonian_noise=True
  algorithm=SVD
  output_dimension=None
  centre=None


/Users/macark/hyperspy-bundle/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "
/Users/macark/hyperspy-bundle/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


Decomposition info:
  normalize_poissonian_noise=True
  algorithm=NMF
  output_dimension=7
  centre=None
scikit-learn estimator:
NMF(n_components=7)


/Users/macark/hyperspy-bundle/lib/python3.8/site-packages/hyperspy/misc/eds/utils.py:627: RuntimeWarning: invalid value encountered in true_divide
  composition = number_of_atoms / total_atoms


191-441 quantified


In [233]:
Pt_counts = hs.signals.Signal2D(Pt_counts)
Pt_counts.change_dtype('float32')
Ni_counts = hs.signals.Signal2D(Ni_counts)
Ni_counts.change_dtype('float32')
 
Pt_counts.save(save_folder+'/1534_Pt_counts.hspy')
Pt_counts.save(save_folder+'/1534_Pt_counts.tif')
Ni_counts.save(save_folder+'/1534_Ni_counts.hspy')
Ni_counts.save(save_folder+'/1534_Ni_counts.tif')

Overwrite '/Users/macark/Desktop/2020.09.02_ChemiSTEM_LATPheating/1247/1247_Al_counts.hspy' (y/n)?
y
Overwrite '/Users/macark/Desktop/2020.09.02_ChemiSTEM_LATPheating/1247/1247_Al_counts.tif' (y/n)?
y
Overwrite '/Users/macark/Desktop/2020.09.02_ChemiSTEM_LATPheating/1247/1247_C_counts.hspy' (y/n)?
y
Overwrite '/Users/macark/Desktop/2020.09.02_ChemiSTEM_LATPheating/1247/1247_C_counts.tif' (y/n)?
y
Overwrite '/Users/macark/Desktop/2020.09.02_ChemiSTEM_LATPheating/1247/1247_O_counts.hspy' (y/n)?
y
Overwrite '/Users/macark/Desktop/2020.09.02_ChemiSTEM_LATPheating/1247/1247_O_counts.tif' (y/n)?
y
Overwrite '/Users/macark/Desktop/2020.09.02_ChemiSTEM_LATPheating/1247/1247_P_counts.hspy' (y/n)?
y
Overwrite '/Users/macark/Desktop/2020.09.02_ChemiSTEM_LATPheating/1247/1247_P_counts.tif' (y/n)?
y
Overwrite '/Users/macark/Desktop/2020.09.02_ChemiSTEM_LATPheating/1247/1247_Ti_counts.hspy' (y/n)?
y
Overwrite '/Users/macark/Desktop/2020.09.02_ChemiSTEM_LATPheating/1247/1247_Ti_counts.tif' (y/n)?
y


In [ ]:
def open_and_save_maps(file_location, 
                       file_name, 
                       start_window, 
                       end_window, 
                       ADF_location,
                       beam_current,
                       elements,
                       lines,
                       save,
                       no_of_components,
                       bin_scale,
                       iw=None, 
                       bw=None):

    """
    This function imports and data set and then saves the ADF, and 
    intensity maps. Whilst returning intensity maps for further use.
    """
    #Import data set in given window range.
    s = hs.load(file_name, first_frame=start_window, last_frame=end_window, load_SI_image_stack=True)
    if save==True:
        #Save ADF image.
        short_name = str(start_window)+'-'+str(end_window)
        adf = s[ADF_location].inav[start_window:end_window]
        adf = adf.rebin(scale=(end_window-start_window,1, 1)).squeeze()
        Image.fromarray(adf.data).save(file_location + '/'+ short_name +'_adf.tiff')
    
    #Extract and rebin EDX data cube.
    s = s[-1]
    s.set_microscope_parameters(beam_current = beam_current, live_time = float(s.original_metadata.Scan.DwellTime)*float(end_window-start_window))
    s= s.rebin(scale=bin_scale)
    s = s.isig[0.1:]

    
    #Decompose EDX
    s.decomposition()
    s.plot_explained_variance_ratio()
    s.decomposition(algorithm='NMF', output_dimension=7)
    #s.plot_decomposition_loadings(no_of_components)
    sc = s.get_decomposition_model(no_of_components)
    sc.add_elements(elements)
    sc.set_lines(lines)
    
    #Set up for intensity extraction
    if bw==None:
        bw = sc.estimate_background_windows(line_width=[3., 3.])
    if iw==None:
        iw = sc.estimate_integration_windows(windows_width=1)
    
    maps = sc.get_lines_intensity()

    return sc, maps

In [79]:
def quantify_and_save(EDXSeries, 
                      s, 
                      maps, 
                      cs, 
                      start_window, 
                      end_window, 
                      threshold, 
                      save=True):
    """
    This function imports and data set and then saves the ADF, and 
    intensity maps. Whilst returning intensity maps for further use.
    """
    
    #Quantify
    results = s.quantification(maps, 
                               method='cross_section',
                               factors = cs,
                               absorption_correction=False)
    
    #Create background cancelled maps.
    #results[1][2].plot()
    binary = (((results[1][0] + results[1][1] + results[1][2] + results[1][3]))>threshold)
    #binary.plot()
    Ni_comp = (results[0][0] + results[0][1]) * binary 
    Pt_comp = (results[0][2] + results[0][3]) * binary 
    Ni_counts = (results[1][0] + results[1][1]) * binary 
    Pt_counts = (results[1][2] + results[1][3]) * binary 
    short_name = str(start_window)+'-'+str(end_window)
    print(short_name, 'quantified')
    
    if save == True:
        hs.plot.plot_images([Ni_comp, Pt_comp], scalebar='all', cmap='viridis', #colorbar='single',
                   centre_colormap='False', tight_layout=True,
                   label=['Ni','Pt'],
                   axes_decor='off',vmin=0)
        
        #matplotlib.pyplot.savefig(EDXSeries + '/' + short_name + '_compound2.tif')
        
        Image.fromarray(Ni_counts.data).save(EDXSeries + '/' + short_name+'_Ni_atoms.tiff')
        Image.fromarray(Pt_counts.data).save(EDXSeries + '/' + short_name+'_Pt_atoms.tiff')
        Image.fromarray(Ni_comp.data).save(EDXSeries + '/' + short_name+'_Ni_composition.tiff')
        Image.fromarray(Pt_comp.data).save(EDXSeries + '/' + short_name+'_Pt_composition.tiff')

    return (Ni_comp, Pt_comp, Ni_counts, Pt_counts)